In [8]:
import torch
import non_local_boxes
import numpy as np

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Definitions

In [9]:
def projection(T):
    T = torch.max(T, torch.zeros_like(T))
    c = torch.sum(T)
    if c>1:  T += -torch.ones_like(T)*(c-1)/T.size(dim=0)
    return T

def loss(T, Q, R):
    n = len(T)
    NewBox = torch.zeros_like(R[0])
    for j in range(n):
        NewBox += T[j]*R[j]   # ! EASY to vectorialize if we need to go faster
    NewBox += (1-torch.sum(T))*R[n]
    return torch.sum(torch.abs(NewBox - Q))

def find_coeff(Q, R, learning_rate, number_steps, do_a_projection=True):
    # Goal: find (a_0, ..., a_{n-1}) such that Q = a_0*R_0 + ... + a_{n-1}*_R_{n-1} + (1-a_0-...-a_{n-1})*R_n
    # How: gradient descent
    # Q is 4x4 matrices
    # R is a list of 4x4 matrices
    # n := len(R)-1
    # T will be torch.tensor([a_0, ..., a_{n-1}])

    T = 0.5 * torch.ones(len(R) - 1)
    T.requires_grad=True
    for i in range(number_steps):
        loss(T, Q, R).backward()
        if do_a_projection:   T = projection(T - T.grad*learning_rate/(i+1)).detach()
        else:   T = (T - T.grad*learning_rate/(i+1)).detach()
        T.requires_grad = True
    return T

In [10]:
def write_combinaison(P, Pname, Boxes, learning_rate=1, number_steps=5000, decimals=2, do_a_projection=True, print_mistakes=False):
    R = [Boxes[j][0] for j in range(len(Boxes))]
    string = Pname+" = "

    T = find_coeff(P.clone(), R, learning_rate, number_steps, do_a_projection)
    values = np.around(T.tolist(), decimals=decimals).tolist()
    values.append(1-sum(values))

    for j in range(len(values)):
        if values[j] != 0:
            if string != Pname+" = ": string += " + "
            if values[j]==1: string += Boxes[j][1]
            else: string += str(values[j])+"·"+Boxes[j][1]
    error = float(loss(torch.tensor(values[:-1]),P,R))
    if error == 0:   string += "    (exact)"
    else:      
        string += "    !!!!! ERROR="+str(error)+" !!!!!"
        if print_mistakes:   print(P)

    print(string)

In [11]:
def multiplication_table(Boxes, W, learning_rate=1, number_steps=5000, decimals=2, do_a_projection=True):
    for Q1 in Boxes:
        print("\n-----\n")
        for Q2 in Boxes:
            P = non_local_boxes.utils.tensor_to_matrix(non_local_boxes.evaluate.R(W, Q1[0], Q2[0]))
            Pname = Q1[1]+" ⊠_W "+Q2[1]
            write_combinaison(P, Pname, Boxes, learning_rate, number_steps, decimals, do_a_projection)
            

In [12]:
P_NL = non_local_boxes.utils.P_NL
P_L = non_local_boxes.utils.P_L

Boxes_all = []  # it will contain all the extremal boxes of NS
for mu in range(2):
    for nu in range(2):
        for sigma in range(2):
            Boxes_all.append([P_NL(mu, nu, sigma), "PNL("+str(mu)+str(nu)+str(sigma)+")"])
            for tau in range(2):
                Boxes_all.append([P_L(mu, nu, sigma, tau), "PL("+str(mu)+str(nu)+str(sigma)+str(tau)+")"])

R_all = [Boxes_all[j][0] for j in range(len(Boxes_all))]

# Draw the Multiplication Table

In [13]:
PR = non_local_boxes.utils.PR
P0 = non_local_boxes.utils.P_0
P1 = non_local_boxes.utils.P_1
I = non_local_boxes.utils.I

In [28]:
Boxes = [
    [PR, "PR"],
    [P_L(0,0,0,0), "P0"],
    #[P_L(0,1,0,1), "P1"],
    [P_L(0,1,1,1), "PL0111"],
    #[I, "I"],
    #[P_L(0,0,0,1), "P01"],
    #[P_L(0,1,0,0), "P10"]
    ]
#W = non_local_boxes.utils.W_BS09(1).detach()
n=1
W = non_local_boxes.utils.W9(n).detach()
W = torch.tensor([1.,0.,0.,1.,0.,0.,1.,0.,1.,1.,0.,0.,0.,0.,1.,1.,1.,0.,0.,1.,1.,0.,0.,1.,1.,0.,0.,1.,0.,1.,1.,0.])*1.
W = torch.t(W.repeat(n, 1))

multiplication_table(
    Boxes=Boxes,
    W = W,
    learning_rate=1,
    number_steps=5000,
    decimals = 2,
    do_a_projection = False
    )


-----

PR ⊠_W PR = PR    (exact)
PR ⊠_W P0 = PR    (exact)
PR ⊠_W PL0111 = PR    (exact)

-----

P0 ⊠_W PR = PR    (exact)
P0 ⊠_W P0 = PL0111    (exact)
P0 ⊠_W PL0111 = P0    (exact)

-----

PL0111 ⊠_W PR = 0.5·P0 + 0.5·PL0111    (exact)
PL0111 ⊠_W P0 = P0    (exact)
PL0111 ⊠_W PL0111 = PL0111    (exact)


---

# Other Stuff...

In [27]:
# W = non_local_boxes.utils.W_NSSRRB22(1).detach()
# non_local_boxes.utils.wiring_to_functions(W)

In [86]:
Boxes = [
    [PR, "PR"],
    [P_L(0,0,0,0), "P0"],
    [P_L(0,1,0,1), "P1"],
    [P_L(0,0,0,1), "P01"],
    [P_L(0,1,0,0), "P10"],
    #[I, "I"],
    #[P_NL(0,1,1), "P_NL(011)"]
    ]

W = non_local_boxes.utils.W_NSSRRB22(1).detach()
P =non_local_boxes.utils.tensor_to_matrix(non_local_boxes.evaluate.R(W, I, I))
Pname = "P"

write_combinaison(
    P=P,
    Pname = Pname,
    Boxes=Boxes,
    learning_rate=1,
    number_steps=50000,
    decimals = 3,
    do_a_projection = False,
    print_mistakes = True
    )

P = 0.1875·P0 + 0.1875·P1 + 0.0625·P01 + 0.5625·P10    (exact)


In [11]:
R = non_local_boxes.evaluate.R
tensor_to_matrix = non_local_boxes.utils.tensor_to_matrix

W = non_local_boxes.utils.W_Pierre_1(1).detach()

Q1 = P0
Q2 = P1
Q3 = PR

torch.all(tensor_to_matrix(R(W, Q1, tensor_to_matrix(R(W, Q2, Q3))))
          ==tensor_to_matrix(R(W, tensor_to_matrix(R(W, Q1, Q2)), Q3))
)

tensor(False)

In [10]:
W = non_local_boxes.utils.W_Pierre_1(1).detach()
non_local_boxes.utils.wiring_to_functions(W)

f_1(x,a2) = x
g_1(y,b2) = y
f_2(x,a1) = x
g_2(y,b1) = y
f_3(x,a1,a2) = a1·a2 ⊕ 1.0 
g_3(y,b1,b2) = b1·b2 ⊕ 1.0 


In [9]:
R = non_local_boxes.evaluate.R
tensor_to_matrix = non_local_boxes.utils.tensor_to_matrix

W = non_local_boxes.utils.W_BS09(1).detach()

torch.all(tensor_to_matrix(R(W, I, PR)) == -(1/8)*P0 -(1/8)*P1 +I + (1/4)*PR)

tensor(True)